In [24]:
import psycopg2
import pandas as pd
import datetime

from faker import Faker
fake = Faker()

In [25]:
# connecting to DB
conn = psycopg2.connect(database="medium",
                        user='postgres', 
                        password='password123', 
                        host='127.0.0.1', 
                        port='5432')

# conn.autocommit = True
cursor = conn.cursor()

In [26]:
# loading csv
data = pd.read_csv('medium-2021-articles-and-nums-v3.csv')
data = data.drop(data.columns[[0]], axis=1)
data

,url,title,author,subtitle,claps,responses,reading_time,tag,article_domain,site_id,...,site_read_time_tag_mean,site_read_time_tag_std,title_len,subtitle_len,nickname_len,title_num_amount,subtitle_num_amount,nickname_num_amount,jarg_title_amount,jarg_subtitle_amount
0,https://towardsdatascience.com/making-python-p...,making python program blazingly fast,martin.heinz,let look performance python program see,3300.0,3.0,5.0,0,towardsdatascience.com,0,...,0.000000,0.000000,5,6,1,0,0,0,0,0
1,https://towardsdatascience.com/how-to-be-fancy...,fancy python,dipam44,python trick make life easier,1700.0,12.0,5.0,0,towardsdatascience.com,0,...,5.000000,0.000000,2,5,1,0,0,0,0,0
2,https://uxdesign.cc/how-exactly-do-you-find-in...,exactly find insight qualitative user research,taylornguyen144,visualizing synthesis process,1100.0,3.0,4.0,0,uxdesign.cc,1,...,0.000000,0.000000,6,3,1,0,0,0,0,1
3,https://towardsdatascience.com/from-scratch-to...,scratch search playing data elasticsearch inge...,stanislavprihoda,one pipeline rule,232.0,1.0,9.0,0,towardsdatascience.com,0,...,5.000000,0.000000,7,3,1,0,0,0,2,0
4,https://www.cantorsparadise.com/the-waiting-pa...,waiting paradox intro probability distribution,maikeelisa,much longer wait,859.0,5.0,8.0,0,www.cantorsparadise.com,2,...,0.000000,0.000000,5,3,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155676,https://medium.com/@james.ralph8555/setting-up...,setting grafana influxdb server monitoring,james.ralph8555,none,0.0,0.0,14.0,6,medium.com,3,...,4.757963,3.155090,5,0,1,0,0,0,3,0
155677,https://medium.com/@jimlyas/using-both-firebas...,using firebase analytics huawei analytics kit ...,jimlyas,cara mudah menggunakan,1.0,0.0,3.0,6,medium.com,3,...,4.759780,3.157439,8,3,1,0,0,0,3,3
155678,https://medium.com/@payodatechnologyinc/the-se...,seven step process make instagram marketing st...,payodatechnologyinc,none,0.0,0.0,6.0,6,medium.com,3,...,4.759434,3.157225,8,0,1,0,0,0,1,0
155679,https://medium.com/@vivek.yadav_256/intermedia...,intermediate introduction solution including p...,vivek.yadav_256,none,137.0,0.0,5.0,6,medium.com,3,...,4.759678,3.156963,6,0,1,0,0,0,1,0


In [27]:
# Normalising Data

# populating domain table
try:
    insert_into_domain_query = """INSERT INTO domains(domain_name) VALUES(%s)"""
    domain_list = [(str(val),) for val in data['article_domain'].unique().tolist()]
    cursor.executemany(insert_into_domain_query, domain_list)
    conn.commit()
except (Exception, psycopg2.DatabaseError) as error:
    print(error)

In [28]:
# populating author table
try:
    insert_into_author_query = """INSERT INTO authors(author, join_date) VALUES(%s, %s)"""
    start_date = datetime.date(year=2018, month=1, day=1)
    end_date = datetime.date(year=2020, month=12, day=31)
    author_list = [(str(val), fake.date_between(start_date=start_date, end_date=end_date),) for val in data['author'].unique().tolist()]
    cursor.executemany(insert_into_author_query, author_list)
    conn.commit()
except (Exception, psycopg2.DatabaseError) as error:
    print(error)

In [29]:
def get_author_id(author_name):
    cursor.execute(f'''SELECT * FROM authors Where author='{author_name}';''')
    return cursor.fetchone()[0]

def get_domain_id(domain_name):
    cursor.execute(f'''SELECT * FROM domains Where domain_name='{domain_name}';''')
    return cursor.fetchone()[0]

def get_article_id(article_title, author_name, domain_name, url):
    cursor.execute(f'''SELECT article_id FROM articles Where title='{article_title}' and author_id={get_author_id(author_name)} and domain_id={get_domain_id(domain_name)} and url='{url}';''')
    return cursor.fetchone()[0]

In [30]:
# populating articles table
try:
    insert_into_articles_query = """INSERT INTO articles(author_id, title, subtitle, url, date_posted, domain_id) VALUES(%s, %s, %s, %s, %s, %s)"""
    
    article_list = []
    for index, row in data.iterrows():
        article_list.append((get_author_id(row['author']), row['title'], row['subtitle'], row['url'], datetime.date(row['year'], row['month'], row['day']), get_domain_id(row['article_domain']),))
    
    cursor.executemany(insert_into_articles_query, article_list)
    conn.commit()
except (Exception, psycopg2.DatabaseError) as error:
    print(error)

In [31]:
# populating article meta table
try:
    insert_into_article_meta_query = """INSERT INTO article_meta(article_id, claps, responses, reading_time, tags, title_len, subtitle_len, prev_mon_topic_rate) VALUES(%s, %s, %s, %s, %s, %s, %s, %s)"""
    
    article_meta_list = []
    for index, row in data.iterrows():
        article_meta_list.append((get_article_id(row['title'], row['author'], row['article_domain'], row['url']), int(row['claps']), int(row['responses']), round(row['reading_time']), int(row['tag']), int(row['title_len']), int(row['subtitle_len']), float(row['prev_mon_topic_rate']),))
    
    cursor.executemany(insert_into_article_meta_query, article_meta_list)
    conn.commit()
except (Exception, psycopg2.DatabaseError) as error:
    print(error)

In [32]:
# populating author trend table
try:
    insert_into_author_trend_query = """INSERT INTO author_trend(author_id, clap_sum, response_sum, read_time_sum) VALUES(%s, %s, %s, %s)"""
    
    cursor.execute('''SELECT author_id, SUM(claps) as total_claps, SUM(responses) as total_responses, SUM(reading_time) as total_reading_time FROM articles, article_meta WHERE articles.article_id = article_meta.article_id GROUP BY author_id;''')
    author_trend_list = cursor.fetchall()    
    cursor.executemany(insert_into_author_trend_query, author_trend_list)
    conn.commit()
except (Exception, psycopg2.DatabaseError) as error:
    print(error)

In [33]:
# populating site trend table
try:
    insert_into_site_trend_query = """INSERT INTO site_trend(domain_id, clap_sum, response_sum, read_time_sum) VALUES(%s, %s, %s, %s)"""
    
    cursor.execute('''SELECT domain_id, SUM(claps) as total_claps, SUM(responses) as total_responses, SUM(reading_time) as total_reading_time FROM articles, article_meta WHERE articles.article_id = article_meta.article_id GROUP BY domain_id;''')
    site_trend_list = cursor.fetchall()    
    cursor.executemany(insert_into_site_trend_query, site_trend_list)
    conn.commit()
except (Exception, psycopg2.DatabaseError) as error:
    print(error)

In [23]:
conn.close()